In [ ]:
import os
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import shutil
from glob import glob
import pandas_profiling as pp
import cv2
from google.cloud import storage
from kaggle_datasets import KaggleDatasets
from random import seed, randint, random, choice
from PIL import Image
import tensorflow_addons as tfa
import sys
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
import gc
import math
import argparse
import warnings
import collections
import tensorflow.keras.backend as K

warnings.filterwarnings("ignore")
print("Tensorflow version " + tf.__version__)

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)
        
        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):

        img = tf.image.random_crop(img, [im_size, im_size, 3])

        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        k90 = randint(0, 3)
        img = tf.image.rot90(img, k=k90)
        
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 12
GCS_DS_PATH = KaggleDatasets().get_gcs_path('plant-pathology-2021-fgvc8')
print('BATCH SIZE:', BATCH_SIZE)

In [ ]:
def swish(x):
    return x * tf.nn.sigmoid(x)

def SEBlock(input_filters,se_output_filters,se_ratio=0.25):
    def block(inputs):
        num_reduced_filters = max(1, int(input_filters * se_ratio))
        x = inputs
        x = tf.keras.layers.Lambda(lambda a: K.mean(a, axis=[1, 2], keepdims=True))(x)
        x = tf.keras.layers.Conv2D(num_reduced_filters,kernel_size=(1, 1),padding='same')(x)
        x = swish(x)
        x = tf.keras.layers.Conv2D(se_output_filters,kernel_size=(1, 1),padding='same',)(x)
        x = tf.keras.layers.Activation('sigmoid')(x)
        out = tf.keras.layers.Multiply()([x, inputs])
        return out
    return block

class DropConnect(tf.keras.layers.Layer):
    def __init__(self, drop_connect_rate=0.):
        super().__init__()
        self.drop_connect_rate = drop_connect_rate

    def call(self, inputs, training=None):
        def drop_connect():
            survival_prob = 1.0 - self.drop_connect_rate
            batch_size = tf.shape(inputs)[0]
            random_tensor = survival_prob
            random_tensor += tf.random_uniform([batch_size, 1, 1, 1], dtype=inputs.dtype)
            binary_tensor = tf.floor(random_tensor)
            output = tf.div(inputs, survival_prob) * binary_tensor
            return output
        return K.in_train_phase(drop_connect, inputs, training=training)

def MBConvBlock(input_filters, output_filters,kernel_size, strides,expand_ratio,drop_connect_rate):
    def block(inputs):
        se_output_filters = input_filters * expand_ratio
        if expand_ratio != 1:
            x = tf.keras.layers.Conv2D(se_output_filters,kernel_size=(1, 1),padding='same',use_bias=False)(inputs)
            x = tf.keras.layers.BatchNormalization()(x)
            x = swish(x)
        else:
            x = inputs
        x = tf.keras.layers.DepthwiseConv2D((kernel_size, kernel_size),strides=strides,padding='same',use_bias=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = swish(x)
        x = SEBlock(input_filters,se_output_filters)(x)
        x = tf.keras.layers.Conv2D(output_filters,kernel_size=(1, 1),padding='same',use_bias=False)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        if all(s == 1 for s in strides) and (input_filters == output_filters):
            if drop_connect_rate:
                x = DropConnect(drop_connect_rate)(x)

            x = tf.keras.layers.Add()([x, inputs])
        return x
    return block


BlockArgs = collections.namedtuple('BlockArgs', ['kernel_size', 'num_repeat', 'input_filters', 'output_filters','expand_ratio', 'strides'])
block_args_list = [
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16,expand_ratio=1, strides=[1, 1]),
    BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24,expand_ratio=6, strides=[2, 2]),
    BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, output_filters=40,expand_ratio=6, strides=[2, 2]),
    BlockArgs(kernel_size=3, num_repeat=3, input_filters=40, output_filters=80,expand_ratio=6, strides=[2, 2]),
    BlockArgs(kernel_size=5, num_repeat=3, input_filters=80, output_filters=112,expand_ratio=6, strides=[1, 1]),
    BlockArgs(kernel_size=5, num_repeat=4, input_filters=112, output_filters=192,expand_ratio=6, strides=[2, 2]),
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=192, output_filters=320,expand_ratio=6, strides=[1, 1])
]
stride_count = 5
num_blocks = 16

def EfficientNet(input_shape,classes,width_coefficient: float,depth_coefficient: float,include_top=True,dropout_rate=0.,drop_connect_rate=0.): 
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = inputs
    x = tf.keras.layers.Conv2D(filters=int(32*width_coefficient), kernel_size=(3,3),strides=(2,2),padding='same',use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = swish(x)    
    drop_connect_rate_per_block = drop_connect_rate / float(num_blocks)
    for block_idx, block_args in enumerate(block_args_list):
        my_input_filters = int(block_args.input_filters*width_coefficient)
        my_output_filters = int(block_args.output_filters*width_coefficient)
        my_num_repeat = math.ceil(block_args.num_repeat*depth_coefficient)
        x = MBConvBlock(my_input_filters, my_output_filters,block_args.kernel_size, block_args.strides,block_args.expand_ratio,drop_connect_rate_per_block * block_idx)(x)
        if my_num_repeat > 1:
            my_input_filters = my_output_filters
            my_strides = [1, 1]
        for _ in range(my_num_repeat - 1):
            x = MBConvBlock(my_input_filters, my_output_filters,block_args.kernel_size, my_strides,block_args.expand_ratio,drop_connect_rate_per_block * block_idx)(x)
    x = tf.keras.layers.Conv2D(filters=int(1280*width_coefficient),kernel_size=(1, 1),padding='same',use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = swish(x)
    if include_top:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        if dropout_rate > 0:
            x = tf.keras.layers.Dropout(dropout_rate)(x)
        x = tf.keras.layers.Dense(classes)(x)
        x = tf.keras.layers.Activation('softmax')(x)
    outputs = x
    model = tf.keras.models.Model(inputs, outputs)
    return model

def EfficientNetB0(input_shape,classes,include_top=True,dropout_rate=0.4,drop_connect_rate=0.): 
    return  EfficientNet(input_shape,classes,1.0,1.0,include_top=include_top,dropout_rate=dropout_rate,drop_connect_rate=drop_connect_rate)

def EfficientNetB1(input_shape,classes,include_top=True,dropout_rate=0.2,drop_connect_rate=0.): 
    return  EfficientNet(input_shape,classes,1.0,1.1,include_top=include_top,dropout_rate=dropout_rate,drop_connect_rate=drop_connect_rate)

def EfficientNetB2(input_shape,classes,include_top=True,dropout_rate=0.3,drop_connect_rate=0.): 
    return  EfficientNet(input_shape,classes,1.1,1.2,include_top=include_top,dropout_rate=dropout_rate,drop_connect_rate=drop_connect_rate)

def EfficientNetB3(input_shape,classes,include_top=True,dropout_rate=0.3,drop_connect_rate=0.): 
    return  EfficientNet(input_shape,classes,1.2,1.4,include_top=include_top,dropout_rate=dropout_rate,drop_connect_rate=drop_connect_rate)

def EfficientNetB4(input_shape,classes,include_top=True,dropout_rate=0.4,drop_connect_rate=0.): 
    return  EfficientNet(input_shape,classes,1.4,1.8,include_top=include_top,dropout_rate=dropout_rate,drop_connect_rate=drop_connect_rate)

def EfficientNetB5(input_shape,classes,include_top=True,dropout_rate=0.4,drop_connect_rate=0.): 
    return  EfficientNet(input_shape,classes,1.6,2.2,include_top=include_top,dropout_rate=dropout_rate,drop_connect_rate=drop_connect_rate)

def EfficientNetB6(input_shape,classes,include_top=True,dropout_rate=0.5,drop_connect_rate=0.): 
    return  EfficientNet(input_shape,classes,1.8,2.6,include_top=include_top,dropout_rate=dropout_rate,drop_connect_rate=drop_connect_rate)

def EfficientNetB7(input_shape,classes,include_top=True,dropout_rate=0.5,drop_connect_rate=0.): 
    return  EfficientNet(input_shape,classes,2.0,3.1,include_top=include_top,dropout_rate=dropout_rate,drop_connect_rate=drop_connect_rate)

def EfficientNetB8(input_shape,classes,include_top=True,dropout_rate=0.5,drop_connect_rate=0.): 
    return  EfficientNet(input_shape,classes,2.2,3.6,include_top=include_top,dropout_rate=dropout_rate,drop_connect_rate=drop_connect_rate)

In [ ]:
IMSIZES = (224, 240, 260, 300, 380, 456, 528, 600, 672, 800)
im_size = IMSIZES[8]

zoom_ratio = 1.2
im_size_zoom = int(im_size * zoom_ratio)
print(im_size, im_size_zoom)

df = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')    
# df = df.iloc[::-1]
# df = df.reset_index(drop=True)

class_name = df.labels.unique().tolist()
class_name.sort()
class_num = len(class_name)

n_labels = 5

print(class_name) 
print(class_num)

img_per_class = []
for c in class_name:
    img_per_class.append(df[df.labels==c])

one_hot = {'healthy':                  [0, 0, 0, 0, 0],
           'complex':                  [1, 0, 0, 0, 0],
           'scab':                     [0, 1, 0, 0, 0],
           'frog_eye_leaf_spot':       [0, 0, 1, 0, 0],
           'rust':                     [0, 0, 0, 1, 0],
           'powdery_mildew':           [0, 0, 0, 0, 1],
           'rust frog_eye_leaf_spot':         [0, 0, 1, 1, 0],
           'scab frog_eye_leaf_spot':         [0, 1, 1, 0, 0],
           'frog_eye_leaf_spot complex':      [1, 0, 1, 0, 0],
           'scab frog_eye_leaf_spot complex': [1, 1, 1, 0, 0],
           'powdery_mildew complex':          [1, 0, 0, 0, 1],
           'rust complex':                    [1, 0, 0, 1, 0]
         }
    
df

In [ ]:
loop = [1] * class_num

train_paths = []
valid_paths = []
train_labels = []
valid_labels = []

TESTSIZE = 0.1
LIMIT = 5000

for i in range(class_num):
    fullsize = img_per_class[i].shape[0]
    print(str(class_name[i]) + ': ' + str(fullsize))
    fullsize = min(fullsize, LIMIT)
    size = int(fullsize * TESTSIZE)
    cnt = 0
    for j in range(loop[i]):
        valid_paths += list(GCS_DS_PATH + '/train_images/' + img_per_class[i][:size]['image'])
        train_paths += list(GCS_DS_PATH + '/train_images/' + img_per_class[i][size:fullsize]['image'])
        valid_labels += list(img_per_class[i][:size]['labels'])
        train_labels += list(img_per_class[i][size:fullsize]['labels'])
        cnt += len(list(GCS_DS_PATH + '/train_images/' + img_per_class[i][size:fullsize]['image']))
    print(str(class_name[i]) + '_train: ' + str(cnt))
    
valid_labels = [one_hot[x] for x in valid_labels]
train_labels = [one_hot[x] for x in train_labels]

print()
print('Train: ', len(train_paths))
print('Valid: ', len(valid_paths))
print('Sum:', len(train_paths) + len(valid_paths))

In [ ]:
train_decoder = build_decoder(with_labels=True, target_size=(im_size_zoom, im_size_zoom))
valid_decoder = build_decoder(with_labels=True, target_size=(im_size, im_size))

train_dataset = build_dataset(
    train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=train_decoder
)

valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=valid_decoder,
    repeat=False, shuffle=False, augment=False
)

In [ ]:
# clean up

del df
del img_per_class
del im_size_zoom
del class_num
del class_name

del train_decoder
del valid_decoder

del loop

del valid_paths
del train_labels
del valid_labels
del one_hot

del TESTSIZE
del LIMIT

del i
del j
del c
del fullsize

del GCS_DS_PATH

gc.collect

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, GlobalAveragePooling2D, ReLU, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D, GlobalMaxPooling2D
import tensorflow_addons

with strategy.scope():
    
    base = EfficientNetB8(include_top=False, input_shape=(im_size, im_size, 3), classes=n_labels)
    
    model = Sequential()
    model.add(base)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(n_labels, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=5e-5), 
                  metrics= ['binary_accuracy'])
    
    model.summary()

In [ ]:
model.load_weights('../input/testeffnet/besteff8.h5')

In [ ]:
checkpoint = ModelCheckpoint('besteff8.h5',
                             save_best_only=True,
                             save_weights_only=True,
                             monitor='val_loss',
                             mode='auto',
                             verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='auto',
                              factor=0.2,
                              patience=20,
                              min_lr=1e-6,
                              verbose=1)

# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)

In [ ]:
steps_per_epoch = len(train_paths) // BATCH_SIZE

del train_paths
del base
del strategy
gc.collect

EPOCHS = 70

In [ ]:
his = model.fit(
            train_dataset, 
            epochs=EPOCHS,
            verbose=1,
            callbacks=[checkpoint, reduce_lr],
            steps_per_epoch=steps_per_epoch,
            validation_data=valid_dataset)

model.save_weights('eff8net.h5')

In [ ]:
# df = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')
# df[-50:]

In [ ]:
df = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')

one_hot = {'healthy':            [0, 0, 0, 0, 0],
           'complex':            [1, 0, 0, 0, 0],
           'scab':               [0, 1, 0, 0, 0],
           'frog_eye_leaf_spot': [0, 0, 1, 0, 0],
           'rust':               [0, 0, 0, 1, 0],
           'powdery_mildew':     [0, 0, 0, 0, 1],
           'scab frog_eye_leaf_spot': [0, 1, 1, 0, 0],
           'rust frog_eye_leaf_spot': [0, 0, 1, 1, 0],
           'frog_eye_leaf_spot complex': [1, 0, 1, 0, 0],
           'scab frog_eye_leaf_spot complex': [1, 1, 1, 0, 0],
           'powdery_mildew complex': [1, 0, 0, 0, 1],
           'rust complex': [1, 0, 0, 1, 0]}

x = 'fffe105cf6808292.jpg'
img = cv2.imread(f'/kaggle/input/plant-pathology-2021-fgvc8/train_images/{x}')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (im_size, im_size))
plt.imshow(img)
img = np.array(img)
img = np.reshape(img, (1, im_size, im_size, 3))
print(df[df.image==x].labels.values[0])
print(one_hot[df[df.image==x].labels.values[0]])
print(model.predict(img / 255.0))

del one_hot
del x
del img
del df

gc.collect